# Perform a SQL Server migration assessment

To perform a SQL Server migration assessment:

0. Download the SqlAssessment utility
1. Set up notebook environment
2. Run the assessment action to get the recommended target platform
3. Run the data collection action
4. Run the SKU recommendation action to get the right Azure SQL Database/Managed Instance SKU based on the collected performance data

**1) Set up notebook environment**

In [ ]:
# The working folder where you downloaded the SqlAssessment utility
$workingDirectory = "<folder containing SqlAssessment utility>"
$fileName = "SqlAssessment.exe"

# SQL Server connection string.
$connectionStrings ="Data Source=.;Initial Catalog=master;Integrated Security=True"

# By default, collect the performance data every 30 seconds and saving into the file every 20 iterations.
$perfQueryIntervalInSec = 30
$numberOfIterations = 20

**2) Run the assessment action to get the recommended target platform**

In [ ]:
$args = "Assess --sqlConnectionStrings `"$connectionStrings`""
Start-Process -Wait -FilePath $fileName -ArgumentList $args -WorkingDirectory $workingDirectory
$json = Get-Content $workingDirectory\SqlAssessmentReport.json | ConvertFrom-Json 

# Show high level target platform recommendations, for more detail please look at SqlAssessmentReport.json in the output folder
$json.Servers.TargetReadinesses.AzureSqlDatabase | Select-Object  -Property AppliesToMigrationTargetPlatform, DatabasesListReadyForMigration, RecommendationStatus
$json.Servers.TargetReadinesses.AzureSqlManagedInstance | Select-Object  -Property AppliesToMigrationTargetPlatform, DatabasesListReadyForMigration, RecommendationStatus

**3) Run the data collection action**

     You can end the collection process by pressing the Enter key or by closing the collection process. 

     Keep the process running for longer durations, preferably during both peak and off-peak hours, to improve recommendation accuracy.

In [ ]:
$args =  "PerfDataCollection --outputFolder $outputFolder  --sqlConnectionStrings `"$connectionStrings`" --perfQueryIntervalInSec $perfQueryIntervalInSec --numberOfIterations $numberOfIterations"
Start-Process -FilePath $fileName -ArgumentList $args -WorkingDirectory $workingDirectory

**4) Run the SKU recommendation action to get the right Azure SQL Database/Managed Instance SKU based on the collected performance data**

In [ ]:
# Select your desired target platform here
$targetPlatform = "AzureSqlDatabase"
# $targetPlatform = "AzureSqlManagedInstance"
# $targetPlatform = "AzureSqlVirtualMachine"

$args= "GetSkuRecommendation --targetPlatform $targetPlatform --perfQueryIntervalInSec $perfQueryIntervalInSec"

Start-Process -Wait -FilePath $fileName -ArgumentList $args -WorkingDirectory $workingDirectory  

# Show high level target SKU recommendations, for more detail + reasoning please look at SkuRecommendationReport.json in the output folder
$json = Get-Content $workingDirectory\SkuRecommendationReport.json |  ConvertFrom-Json  
$json | Select-Object -ExpandProperty RecommendedSku -Property SqlInstanceName, DatabaseName                                 